<h2><center>Submission by Pranov Suresh</center></h2>

### Question 1

In [1]:
#Importing essential packages and modules
import wooldridge as woo
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats
import statsmodels.formula.api as smf
import statsmodels.api as sm
import numpy as np
import statsmodels.stats.outliers_influence as smo
import patsy as pt

In [2]:
#Importing data from wooldridge module as discrim
discrim = woo.data("DISCRIM")
#Checking for null values in the dataset
print(discrim)
print(discrim.isnull().sum())

     psoda  pfries  pentree  wagest  nmgrs  nregs  hrsopen   emp  psoda2  \
0     1.12    1.06     1.02    4.25    3.0    5.0     16.0  27.5    1.11   
1     1.06    0.91     0.95    4.75    3.0    3.0     16.5  21.5    1.05   
2     1.06    0.91     0.98    4.25    3.0    5.0     18.0  30.0    1.05   
3     1.12    1.02     1.06    5.00    4.0    5.0     16.0  27.5    1.15   
4     1.12     NaN     0.49    5.00    3.0    3.0     16.0   5.0    1.04   
..     ...     ...      ...     ...    ...    ...      ...   ...     ...   
405   1.11    1.05     0.94    4.50    3.0    2.0     12.5  10.0    1.11   
406   0.95    0.74     2.33    4.75    3.0    4.0     10.0   8.0    0.95   
407   0.97    0.84     0.91    4.25    4.0    3.0     18.0  35.0    0.97   
408   0.97    0.86     0.89    4.75    4.0    3.0     17.0  18.5    0.97   
409   1.02    0.89     0.90    4.50    3.0    3.0     15.0  29.0    1.02   

     pfries2  ...  county    lpsoda   lpfries    lhseval    lincome  ldensity  \
0     

Despite the presence of multiple null values in the dataset, in the absence of any guidance regarding the imputation of these, a choice is made to retain these NaNs in the dataset and run all models with them. Output is influenced by the presence of these NaNs.

In [3]:
#Defining variables to be used in the model
proportion_black = discrim["prpblck"]
price_soda = np.log(discrim["psoda"])
log_income = np.log(discrim["income"])
poverty = discrim["prppov"]

#Creating the regression model
reg = smf.ols("price_soda~proportion_black+log_income+poverty", data = discrim).fit()
print(reg.summary())

                            OLS Regression Results                            
Dep. Variable:             price_soda   R-squared:                       0.087
Model:                            OLS   Adj. R-squared:                  0.080
Method:                 Least Squares   F-statistic:                     12.60
Date:                Thu, 30 Nov 2023   Prob (F-statistic):           6.92e-08
Time:                        20:35:31   Log-Likelihood:                 439.04
No. Observations:                 401   AIC:                            -870.1
Df Residuals:                     397   BIC:                            -854.1
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept           -1.4633      0.294  

In [4]:
#Estimating the confidence intervals of the estimates at both the 95% and 99% confidence levels
print(reg.conf_int(alpha=0.05))
print(reg.conf_int(alpha=0.01))

                         0         1
Intercept        -2.040755 -0.885909
proportion_black  0.012500  0.133114
log_income        0.084355  0.189555
poverty           0.119300  0.641420
                         0         1
Intercept        -2.223535 -0.703129
proportion_black -0.006589  0.152204
log_income        0.067705  0.206206
poverty           0.036663  0.724057


Based on the summary of the regression model built, and the additional confidence interval constructed for the coefficient of the "proportion_black" variable, it is clear that at the 95% confidence level B1 is not equal to zero. There are two approaches to arriving at this conclusion. First, we look at the T-value and p-value of the T-test conducted on the two-sided Null Hypothesis that B1 = 0. For a 95% confidence level, the absolute value of the two-sided T-value must exceed 1.96 and the p-value must not exceed 0.05. Second, we can look at the confidence interval constructed for the coefficient using a 95% confidence level. Here the interval [0.012:0.133] does not include the value 0. Since both these conditions are satisfied by the model, we can conclude that H0: B1 = 0 can be rejected at the 95% confidence level. 

For the 99% confidence level, we use the same two tests but modified for a higher significance level. Here, the T-value and p-value for the 99% confidence level are 2.576 and 0.01 respectively. The confidence interval constructed for the 99% confidence level is [-0.006:0.152]. Based on both these findings, we can clearly see that 0 is a feasible value for B1 at the 99% confidence level. Thus, we can conclude that H0: B1 = 0 cannot be rejected, and B1 is not statistically different from 0 at the 99% confidence level. 

In [5]:
#Checking for correlation between the predictors
correlation_list = ["prppov", "prpblck", "lpsoda", "lincome"]
correlation_table = discrim[correlation_list].corr()
print(correlation_table)

           prppov   prpblck    lpsoda   lincome
prppov   1.000000  0.680283  0.024875 -0.838467
prpblck  0.680283  1.000000  0.135393 -0.496636
lpsoda   0.024875  0.135393  1.000000  0.126000
lincome -0.838467 -0.496636  0.126000  1.000000


Here, we have the correlation matrix for all the variables involved in the model. The correlation coefficient between log income and the proportion of people who are below the poverty line is -0.8384. This is in line with the standard economic intuition that a rise in the average income in the region will naturally mean that the proportion of people in poverty in the region is lower. 

In [6]:
#Creating patsy matrices to estimate VIF
y, X = pt.dmatrices('price_soda~log_income+poverty+proportion_black', data=discrim, return_type='dataframe')
K = X.shape[1]
VIF = np.empty(K)
for i in range(K):
    VIF[i] = smo.variance_inflation_factor(X.values, i)

#Calculating VIF
print(f'VIF: \n{VIF}\n')

VIF: 
[5.22495302e+03 3.51872083e+00 4.91521981e+00 1.92215952e+00]



The VIF test for multicollinearity returns values here that are range bound between 1 and 5. The typical interpretation of VIF results below 5, but not equal to 1 is that mild multicollinearity exists in the model. Here, we know that the income and poverty indicators are bound to be linked, as they are economically connected. When we created the correlation matrix, that intuition was confirmed with a high correlation coefficient of -0.8384. Despite this, we find that this model does not have a high enough VIF to reject its significance entirely, and look to build a new model. 

In [7]:
#Introducing new variable into the model
log_house_value= discrim["lhseval"]

#Building new model
reg2 = smf.ols("price_soda~log_income+proportion_black+poverty+log_house_value", data=discrim).fit()
print(reg2.summary())
print(reg2.conf_int(alpha=0.01))

                            OLS Regression Results                            
Dep. Variable:             price_soda   R-squared:                       0.184
Model:                            OLS   Adj. R-squared:                  0.176
Method:                 Least Squares   F-statistic:                     22.31
Date:                Thu, 30 Nov 2023   Prob (F-statistic):           1.24e-16
Time:                        20:35:34   Log-Likelihood:                 461.55
No. Observations:                 401   AIC:                            -913.1
Df Residuals:                     396   BIC:                            -893.1
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept           -0.8415      0.292  

Here, the model is augmented by the addition of the variable (log_house_value). This gives us insight into average property values in the area, and the potential impact it could have on the price of soda. The coefficient of the log_house_value variable is 0.1213, which can be interpreted as a 12% increase in the log price of soda for a 1 unit increase in the house value in the area. Framing a null hypothesis that B4 = 0, we receive a T-value of 6.860 and a p-value of 0.000, which reveals statistical significance at the 95% confidence level. Thus, we can reject the null hypothesis, and state that B4 is not equal to 0. We can additionally verify this conclusion using the confidence interval constructed for the 95% confidence level, where the interval is [0.0755:0.1670]. Since the value 0 does not lie within these range, we can reject the null hypothesis.

In [8]:
#Formulating the hypothesis test and printing the joint-test F value
print("The statistical significance of the regression coefficient of the proportion of poverty variable  in the original model is", reg.pvalues[3],"\nThe statistical significance of the regression coefficient of the log of income variable  in the original model is", reg.pvalues[2],"\nThe statistical significance of the regression coefficient of the proportion of poverty variable in the augmented model is", reg2.pvalues[3],"\nThe statistical significance of the regression coefficient of the log of income variable in the augmented model is", reg2.pvalues[1])

The statistical significance of the regression coefficient of the proportion of poverty variable  in the original model is 0.004400360031387765 
The statistical significance of the regression coefficient of the log of income variable  in the original model is 4.802040557260754e-07 
The statistical significance of the regression coefficient of the proportion of poverty variable in the augmented model is 0.6985696891508966 
The statistical significance of the regression coefficient of the log of income variable in the augmented model is 0.15870684169836033


Here the statistical significance of the variables as measured by the p-values is clearly declining from the original model to the augmented model. When we add the log of the average house value in an area to the model, the statistical significance of both the income and poverty variables drastically declines. 

For poverty, when the p value goes from 0.0044 to 0.6985, it goes from statistically significant at a 99% confidence level to being insignificant at even a 95% confidence level. 

For the income variable, when the p value goes from 0.000 to 0.1587, it too similarly goes from being statistically significant at 99% confidence level to statistical insignificance at the 95% confidence level. 

In [12]:
hypotheses = ["log_income = 0", "poverty = 0"]
print("The F-test results for the joint hypothesis that both income and proportion of impoverished individuals can explain some of the variance in the price of soda is", reg2.f_test(hypotheses))

The F-test results for the joint hypothesis that both income and proportion of impoverished individuals can explain some of the variance in the price of soda is <F test: F=3.5226857196147807, p=0.030448581905393016, df_denom=396, df_num=2>


The F statistic for the joint hypothesis is 3.5226, with a p-value of 0.03, signifying that these variables are jointly significant with respect to the price of soda variable. However, their statistical significance is clearly impacted above in the augmented model through the introduction of the log_house_value variable. Since the income and poverty variables are economically connected, and intuitively they may also be connected to the housing value variable, they are independently and jointly significant yet lose significance in the augmented model. This means that these variables explain less of the variance in the price of soda when part of a larger model. 

When comparing two multiple regression models, we can rely upon a multitude of indicators and tests to serve as a statistical basis for picking the better model. 
1. First, we can look at the adjusted R square, which explains the quantum of variance in the price of soda that can be explained by the independent variables. In the original model we have an adjusted R sq of 0.080 as compared to the augmented model's 0.176. This tilts the field in the favour of our augmented model, purely on the basis of it being able to explain a greater quantum of variance in our dependent variable.
2. Second, we look at the Akaike and Bayesian Information Criterion (AIC and BIC) which are tests surrounding our choice of variables. Typically, AIC and BIC are used to evaluate if the addition of a variable makes it more statistically valid. The lower the AIC and BIC, the better the selection of variables within a model. Once again, the augmented model here is clearly ahead with much lower AIC and BIC values as compared to the original model.
3. Lastly, we can look at the F-statistic joint hypothesis which examines if the independent variables are jointly signiificant with respect to the predicted variable. Our augmented model has a significantly higher F statistic, with a signficantly higher p value when compared to the original model. This further adds to the argument in favour of it.

To conclude, we can use the above information to argue in favour of the selection of the second model. This is despite the major limitation that two variables within that model are not statistically significant at the 95% confidence level. 

## Question 2

In [13]:
#defining variables to be introduced into the model
vote = pd.DataFrame(woo.data("VOTE1"))
print(vote.describe)
vote1 = vote["voteA"]
president_vote = vote["prtystrA"]
expenditure_A = np.log(vote["expendA"])
party = vote["democA"]
expenditure_B = np.log(vote["expendB"])

#building the model and deriving residuals
reg3 = smf.ols("vote1~president_vote+expenditure_A+party+expenditure_B", data = vote).fit()
print(reg3.summary())
y_hat = reg3.resid

<bound method NDFrame.describe of     state  district  democA  voteA     expendA      expendB  prtystrA  \
0      AL         7       1     68  328.295990     8.737000        41   
1      AK         1       0     62  626.377014   402.476990        60   
2      AZ         2       1     73   99.607002     3.065000        55   
3      AZ         3       0     69  319.690002    26.281000        64   
4      AR         3       0     75  159.220993    60.054001        66   
..    ...       ...     ...    ...         ...          ...       ...   
168    WV         4       0     39   32.039001   152.270996        42   
169    WI         3       1     32   22.625999   359.800995        53   
170    WI         5       0     36  197.460007  1278.526001        36   
171    WI         7       0     38  202.591003   450.716003        46   
172    WI         8       1     30   14.421000   227.822998        47   

     lexpendA  lexpendB     shareA  
0    5.793916  2.167567  97.407669  
1    6.439952  

In [14]:
#Running tests for heteroskedasticity
models_to_run = [president_vote, expenditure_A, party, expenditure_B]
test1 = smf.ols("y_hat~expenditure_A", vote).fit()
test2 = smf.ols("y_hat~president_vote", vote).fit()
test3 = smf.ols("y_hat~party", vote).fit()
test4 = smf.ols("y_hat~expenditure_B", vote).fit()
print(test1.summary())
print(test2.summary())
print(test3.summary())
print(test4.summary())

                            OLS Regression Results                            
Dep. Variable:                  y_hat   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.006
Method:                 Least Squares   F-statistic:                 3.228e-14
Date:                Thu, 30 Nov 2023   Prob (F-statistic):               1.00
Time:                        20:36:13   Log-Likelihood:                -593.20
No. Observations:                 173   AIC:                             1190.
Df Residuals:                     171   BIC:                             1197.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept     -4.741e-14      1.884  -2.52e-14

Here, when we build the original model and obtain the residuals from it, we are operating under the typical assumptions of the Ordinary Least Squares estimation process which is that the expected value of the error term, u_hat when conditioned upon any particular value of X is 0. Mathematically, this can be stated as E(u|X) = 0. This is also known as the homoskedasticity property, which states that there is no statistical relationship between the error terms and the indicators within the regression model. 

When we regress the residuals U_hat on the indicators, we obtain a R squared of 0.000, which supports the above property that any variance in the residuals cannot be explained by variance in the indicators. This supports the homoskedastic errors assumption. 

In [15]:
#Running the breusch-pagan test for heteroskedasticity. 
y1, X1 = pt.dmatrices('vote1 ~ president_vote+expenditure_A+party+expenditure_B', vote, return_type = 'dataframe')

print("The F-statistic Breusch Pagan test returns",sm.stats.diagnostic.het_breuschpagan(reg3.resid, X1))

The F-statistic Breusch Pagan test returns (9.093355957811157, 0.05880791685694346, 2.3301126837162482, 0.058057514088554946)


When we conduct the Breusch Pagan test for Heteroskedasticity, we take the square of the residuals from the model and regress it against the independent variables from that model. The standard joint hypothesis test using an F-statistic is used to verify if any of the variation in the residual is explained by variation in the independent variables. The F-statistic and the associated p-value reveal that the null hypothesis of heteroskedasticity is rejected. With a p-value of 0.058 at a 95% confidence level, we reject the null hypothesis. Although, the close nature of this result indicates that further testing for heteroskedasticity needs to be done to be confirmed, as done below with the White test. 

In [16]:
print("The F-statistic White test returns",sm.stats.diagnostic.het_white(reg3.resid, X1))

The F-statistic White test returns (31.101516025646724, 0.0032582616087890475, 2.680757782482784, 0.0019733250042553866)


Here, a similar joint hypothesis F-test is conducted to check if the residuals are linked to the independent variables. The only difference is that the White test specifically checks for constant errors, which is in line with homoskedasticity. The null hypothesis formulated is that errors are not consistent. 

With a p-value of 0.001, this is statistically significant at the 95% confidence level and thus, we uphold the null hypothesis. With the White test suggesting heteroskedasticity is present in the data, and the Breusch-Pagan suggesting the opposite, further testing must take place to confirm our findings. The errors are not constant, with variance detected and the Breusch Pagan test being close to statistically significant all point towards mild heteroskedasticity being present in the data. 

In [17]:
robust_reg = smf.ols('vote1 ~ president_vote+expenditure_A+party+expenditure_B', vote).fit(cov_type = 'HC0')
print(robust_reg.summary())

                            OLS Regression Results                            
Dep. Variable:                  vote1   R-squared:                       0.801
Model:                            OLS   Adj. R-squared:                  0.796
Method:                 Least Squares   F-statistic:                     169.1
Date:                Thu, 30 Nov 2023   Prob (F-statistic):           8.58e-58
Time:                        20:36:18   Log-Likelihood:                -593.20
No. Observations:                 173   AIC:                             1196.
Df Residuals:                     168   BIC:                             1212.
Df Model:                           4                                         
Covariance Type:                  HC0                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         37.6614      4.355      8.

When compared to the original model, the coefficients and the results of the statistical significance hypothesis test are all notably the same in the robust standard errors model. This is as expected as the only modification from the standard OLS estimates is that the standard errors are calculated on the basis of the assumption that the data displays heteroskedasticity. 

There is a consistent decrease in the standard error of the estimates of all the coefficients, which is a reflection of this model's better statistical accuracy. The lower the standard error of the estimate, the closer it is to the actual observed value in the population. 

In [18]:
vote["res2"] = np.log(reg3.resid**2)
w_est = smf.ols('res2 ~ president_vote+expenditure_A+party+expenditure_B', data = vote).fit()

vari = np.exp(w_est.fittedvalues)
w = 1/vari

fgls =smf.wls('vote1 ~ president_vote+expenditure_A+party+expenditure_B', vote, weights = w).fit()

print(fgls.summary())

                            WLS Regression Results                            
Dep. Variable:                  vote1   R-squared:                       0.790
Model:                            WLS   Adj. R-squared:                  0.785
Method:                 Least Squares   F-statistic:                     157.9
Date:                Thu, 30 Nov 2023   Prob (F-statistic):           8.08e-56
Time:                        20:36:19   Log-Likelihood:                -591.49
No. Observations:                 173   AIC:                             1193.
Df Residuals:                     168   BIC:                             1209.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         40.7478      4.871      8.

When compared to the original model, the Feasible Generalised Least Squares (FGLS) model displays a change in the estimates as is expected when assigning weights through the integrated WLS method. Despite a slight elevation in the standard errors across the board, they are still relatively tight to the range seen in both the robust standard errors and standard OLS models. What is interesting however, is the slight change in the p-values of the coefficient estimate hypothesis tests which earlier were all significant at the 99% confidence level. Now, party only showcases significance at the 95% confidence level, while president_vote still remains at the previous level albeit with an elevated p-value. 

## Question 3

In [19]:
injury = woo.data("injury")
print(injury.dtypes)
log_benefit_time = injury["ldurat"]
income = injury["highearn"]
gender = injury["male"]
maritial_status = injury["married"]
age = injury["age"]
injury_type = injury["injtype"]
benefit_change = injury["afchnge"]

reg5 = smf.ols("log_benefit_time~income", injury).fit()
print(reg5.summary())

durat       float64
afchnge       int64
highearn      int64
male        float64
married     float64
hosp          int64
indust      float64
injtype       int64
age         float64
prewage     float64
totmed      float64
injdes        int64
benefit     float64
ky            int64
mi            int64
ldurat      float64
afhigh        int64
lprewage    float64
lage        float64
ltotmed     float64
head          int64
neck          int64
upextr        int64
trunk         int64
lowback       int64
lowextr       int64
occdis        int64
manuf       float64
construc    float64
highlpre    float64
dtype: object
                            OLS Regression Results                            
Dep. Variable:       log_benefit_time   R-squared:                       0.013
Model:                            OLS   Adj. R-squared:                  0.013
Method:                 Least Squares   F-statistic:                     95.46
Date:                Thu, 30 Nov 2023   Prob (F-statistic):           

### The above model is the standalone model to which we can compare our controlled model below. 

In [20]:
reg_control = smf.ols("log_benefit_time~income+gender+maritial_status+age+injury_type", injury).fit()
print(reg_control.summary())

                            OLS Regression Results                            
Dep. Variable:       log_benefit_time   R-squared:                       0.025
Model:                            OLS   Adj. R-squared:                  0.024
Method:                 Least Squares   F-statistic:                     34.66
Date:                Thu, 30 Nov 2023   Prob (F-statistic):           4.06e-35
Time:                        20:36:35   Log-Likelihood:                -11434.
No. Observations:                6844   AIC:                         2.288e+04
Df Residuals:                    6838   BIC:                         2.292e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           0.7352      0.073     

In [23]:
reg_control1 = smf.ols("log_benefit_time~income+gender+maritial_status+age+C(injury_type)", injury).fit()
print(reg_control1.summary())

                            OLS Regression Results                            
Dep. Variable:       log_benefit_time   R-squared:                       0.035
Model:                            OLS   Adj. R-squared:                  0.034
Method:                 Least Squares   F-statistic:                     22.61
Date:                Thu, 30 Nov 2023   Prob (F-statistic):           4.02e-46
Time:                        20:40:02   Log-Likelihood:                -11397.
No. Observations:                6844   AIC:                         2.282e+04
Df Residuals:                    6832   BIC:                         2.290e+04
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept               0.4625    

Here in our complete model with additional control variables in the form of gender, maritial status, age, and injury type we see that the coefficient of the income variable has changed to 0.2544. This can be interpreted as the log of benefit duration being higher for high earners within the population set, due to the high statistical significance of the income variable. 

When we build an additional model where we classify the injury type variable as a categorical variable it changes to 0.2470. 

In [24]:
reg_random_assign = smf.ols("income~gender+age+maritial_status+injury_type", injury).fit()
print(reg_random_assign.summary())

                            OLS Regression Results                            
Dep. Variable:                 income   R-squared:                       0.205
Model:                            OLS   Adj. R-squared:                  0.205
Method:                 Least Squares   F-statistic:                     441.0
Date:                Thu, 30 Nov 2023   Prob (F-statistic):               0.00
Time:                        20:41:30   Log-Likelihood:                -4020.9
No. Observations:                6844   AIC:                             8052.
Df Residuals:                    6839   BIC:                             8086.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -0.3063      0.024    -

Once we build our treatment effects model above, we check for random assignment to the treatment group by regressing it against the remaining independent variables. Here our main focus is on two statistical measures, the F-statistic hypothesis for joint significance and the adjusted R square. 

The joint hypothesis test's F-statistic is 441 with a p-value of 0.00, indicating that all the remaining independent variables are significant with respect to our treatment group, income. 
The adjusted R square of 0.205 reveals that a significant 20% of the variation in assignment to the treatment group is caused by variances within the other variables. 

On the basis of this combined evidence, we can clearly confirm that assignment to the treatment group is not random. 

In [25]:
reg_DID = smf.ols("log_benefit_time~maritial_status+age+gender+injury_type+income*benefit_change", injury).fit()
print(reg_DID.summary())

                            OLS Regression Results                            
Dep. Variable:       log_benefit_time   R-squared:                       0.028
Model:                            OLS   Adj. R-squared:                  0.027
Method:                 Least Squares   F-statistic:                     28.01
Date:                Thu, 30 Nov 2023   Prob (F-statistic):           2.90e-38
Time:                        20:41:32   Log-Likelihood:                -11423.
No. Observations:                6844   AIC:                         2.286e+04
Df Residuals:                    6836   BIC:                         2.292e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept                 0.73

#### When building a Differences-in-Differences model, we seek to establish if the duration of benefits are spread differently among different groups within the treatment group. 

In this case, the interaction term we pick consists of the income and benefit_change variables, where the coefficient is estimated to be 0.2049 with a strong statistical significance as reflected by the p-value of 0.001. Thus, we can establish a strong difference in duration of the benefits between high income and low income workers to the tune of 20% in favour of the high income workers. 

## Question 4

In [26]:
#building the linear probability model 
loan_data = woo.data("loanapp")
approval = loan_data["approve"]
white = loan_data["white"]

linear_reg = smf.ols("approval~white", loan_data).fit(cov_type="HC3")
print(linear_reg.summary())

                            OLS Regression Results                            
Dep. Variable:               approval   R-squared:                       0.049
Model:                            OLS   Adj. R-squared:                  0.048
Method:                 Least Squares   F-statistic:                     55.47
Date:                Thu, 30 Nov 2023   Prob (F-statistic):           1.41e-13
Time:                        20:41:35   Log-Likelihood:                -555.54
No. Observations:                1989   AIC:                             1115.
Df Residuals:                    1987   BIC:                             1126.
Df Model:                           1                                         
Covariance Type:                  HC3                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.7078      0.026     27.225      0.0

In [27]:
#Creating predictions using the linear probability model 
X_new = pd.DataFrame({'white': [0,1]})
predictions_linear = linear_reg.predict(X_new)

print(f'predictions: \n{predictions_linear}\n')

predictions: 
0    0.707792
1    0.908388
dtype: float64



In [28]:
#Building the probit model
probit_reg = smf.probit("approval~white", loan_data).fit(disp=0)
print(probit_reg.summary())

#Creating predictions using the probit model
predictions_probit = probit_reg.predict(X_new)
print(predictions_probit)

                          Probit Regression Results                           
Dep. Variable:               approval   No. Observations:                 1989
Model:                         Probit   Df Residuals:                     1987
Method:                           MLE   Df Model:                            1
Date:                Thu, 30 Nov 2023   Pseudo R-squ.:                 0.05331
Time:                        20:41:37   Log-Likelihood:                -700.88
converged:                       True   LL-Null:                       -740.35
Covariance Type:            nonrobust   LLR p-value:                 6.408e-19
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.5469      0.075      7.251      0.000       0.399       0.695
white          0.7839      0.087      9.041      0.000       0.614       0.954
0    0.707792
1    0.908388
dtype: float64


Using the probit and linear probability models, we estimated the probability of inidviduals belonging to the white racial group and other racial groups being approved for a loan. The linear probability model had a intercept estimate of 0.707 and a white coefficient estimate of 0.2006. Based on these parameters, the linear probability model estimated that the approval rates for white people was 0.9083 and for non-whites was 0.7077. 

The probit model also estimated the exact same probabilities for loan approval of 90.83% and 70.77% respectively. With such a large divergence and statistically significant results all around, this deserves attention although neither the linear probability nor probit model can explain causal inference. 

In [29]:
#Extracting variables to add to the augmented probit model 
housing_exp = loan_data["hrat"]
other_exp = loan_data["obrat"]
loan_amount = loan_data["loanprc"]
unemployment = loan_data["unem"]
gender = loan_data["male"]
maritial_status1 = loan_data["married"]
dependents = loan_data["dep"]
schooling = loan_data["sch"]
cosign = loan_data["cosign"]
credit_history = loan_data["chist"]
bankruptcy = loan_data["pubrec"]
late_payment1 = loan_data["mortlat1"]
late_payment2 = loan_data["mortlat2"]
vacancy_rate = loan_data["vr"]

#Building the augmented probit model
probit_reg1 = smf.probit("approval~white+housing_exp+other_exp+loan_amount+unemployment+gender+maritial_status1+dependents+schooling+cosign+credit_history+bankruptcy+late_payment1+late_payment2+vacancy_rate", loan_data).fit(disp=0)
print(probit_reg1.summary())

                          Probit Regression Results                           
Dep. Variable:               approval   No. Observations:                 1971
Model:                         Probit   Df Residuals:                     1955
Method:                           MLE   Df Model:                           15
Date:                Thu, 30 Nov 2023   Pseudo R-squ.:                  0.1866
Time:                        20:41:49   Log-Likelihood:                -600.27
converged:                       True   LL-Null:                       -737.98
Covariance Type:            nonrobust   LLR p-value:                 7.014e-50
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            2.0623      0.313      6.585      0.000       1.449       2.676
white                0.5203      0.097      5.366      0.000       0.330       0.710
housing_exp          0.0079 

With a tight standard error of 0.097, high statistical significance reflected by a p-value of 0.000 and a coefficient of 0.5203, there is enough statistical evidence to suggest that there is significant racial discrimination in the approval of loans, skewed in favour of white individuals. 

The pretty high coefficient of nearly 52% should raise concern over the mortgage loan approval practises. 

In [30]:
#Building the logit model
logit_reg = smf.logit("approval~white+housing_exp+other_exp+loan_amount+unemployment+gender+maritial_status1+dependents+schooling+cosign+credit_history+bankruptcy+late_payment1+late_payment2+vacancy_rate", loan_data).fit(disp=0)
print(logit_reg.summary())

                           Logit Regression Results                           
Dep. Variable:               approval   No. Observations:                 1971
Model:                          Logit   Df Residuals:                     1955
Method:                           MLE   Df Model:                           15
Date:                Thu, 30 Nov 2023   Pseudo R-squ.:                  0.1863
Time:                        20:41:51   Log-Likelihood:                -600.50
converged:                       True   LL-Null:                       -737.98
Covariance Type:            nonrobust   LLR p-value:                 8.693e-50
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            3.8017      0.595      6.393      0.000       2.636       4.967
white                0.9378      0.173      5.424      0.000       0.599       1.277
housing_exp          0.0133 

The logit model continues to confirm the findings of the probit model. With a coefficient of 93.78%, a p-value of 0.000 which signifies high statistical significance, and a less tight standard error of 0.173 the logit model continues to prove that evidence of racial discrimination in the practice of approving mortage loans exists. 

To equalise the base of the coefficients from both models, we divide the logit coefficient by 1.6. Here 0.9378/1.6 gives us 0.586125, which is within close range of our probit model. This confirms that our logit model is statistically valid. 

In [31]:
#Estimating the probit marginal effects
probit_reg1.get_margeff().margeff

array([ 0.08638681,  0.00130785, -0.00459826, -0.16803531, -0.00609144,
       -0.00614401,  0.0441267 , -0.00823193,  0.00243254,  0.01429195,
        0.09718468, -0.12930819, -0.0311545 , -0.08208678, -0.03338594])

In [32]:
#Estimating the logit marginal effects
logit_reg.get_margeff().margeff

array([ 0.08281951,  0.00117134, -0.00468373, -0.16823751, -0.00587998,
       -0.00586287,  0.04444778, -0.00801322,  0.00364115,  0.01166291,
        0.0941957 , -0.11840206, -0.02736752, -0.07901408, -0.03089537])

In [33]:
#Estimating the average partial effects of the probit and logit models
probit_ape = probit_reg1.params*stats.norm.pdf(probit_reg1.fittedvalues).mean()
logit_ape = logit_reg.params*stats.logistic.pdf(logit_reg.fittedvalues).mean()

print("Probit APE: ", probit_ape, "Probit PEA: ", logit_ape)

Probit APE:  Intercept           0.342445
white               0.086387
housing_exp         0.001308
other_exp          -0.004598
loan_amount        -0.168035
unemployment       -0.006091
gender             -0.006144
maritial_status1    0.044127
dependents         -0.008232
schooling           0.002433
cosign              0.014292
credit_history      0.097185
bankruptcy         -0.129308
late_payment1      -0.031155
late_payment2      -0.082087
vacancy_rate       -0.033386
dtype: float64 Probit PEA:  Intercept           0.335751
white               0.082820
housing_exp         0.001171
other_exp          -0.004684
loan_amount        -0.168238
unemployment       -0.005880
gender             -0.005863
maritial_status1    0.044448
dependents         -0.008013
schooling           0.003641
cosign              0.011663
credit_history      0.094196
bankruptcy         -0.118402
late_payment1      -0.027368
late_payment2      -0.079014
vacancy_rate       -0.030895
dtype: float64


The Average Partial Effect isolates the effect of one particular variable and its influence on the conditional probability. Here, the estimate of the APE for the probit model is 0.0863 and the logit model is 0.0828. This means that according to our model, leaving all else equal, being white increases your probability of having your mortgage loan approved by at least 8.2% according to logit or upto 8.63% according to probit. 

This combined with our previous statistically significant results may not be sufficient to build a case of causal inference, exhaustively demonstrating that racial discrimination against non-white individuals occurred. However, it definitely provides evidence to justify further examination of the data. 